In [ ]:
from keras.preprocessing.image import *

from tqdm import tqdm

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
import csv
import cv2

import os
from PIL import Image

from glob import glob

%matplotlib inline
from jupyterthemes import jtplot

jtplot.style()

## Read train images

In [ ]:
images_per_class = {}

for class_label in tqdm(os.listdir('train')):
    class_folder_path = 'train/%s' % class_label
    images_per_class[class_label] = []
    for image_path in glob(os.path.join(class_folder_path, "*.png")):
        image = img_to_array(load_img(image_path, target_size=(224, 224, 3)))
        images_per_class[class_label].append(image)


### Number of images per class

In [ ]:
total = 0
categories = []
for key, value in sorted(images_per_class.items()):
    print("{0} -> {1}".format(key, len(value)))
    total+= len(value)
    categories.append(key)
print('Total Images', total)
print(categories)

### Plot images

In [ ]:
def plot_for_class(label):
    nb_rows = 3
    nb_cols = 3
    fig, axs = plt.subplots(nb_rows, nb_cols, figsize=(6, 6))

    n = 0
    for i in range(0, nb_rows):
        for j in range(0, nb_cols):
            axs[i, j].xaxis.set_ticklabels([])
            axs[i, j].yaxis.set_ticklabels([])
            axs[i, j].imshow(images_per_class[label][n].astype('uint8'))
            n += 1        

In [ ]:
plot_for_class('Fat Hen')

## preprocess images

In [ ]:
def create_mask_for_plant(image):
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    sensitivity = 35
    lower_hsv = np.array([60 - sensitivity, 100, 50])
    upper_hsv = np.array([60 + sensitivity, 255, 255])

    mask = cv2.inRange(image_hsv, lower_hsv, upper_hsv)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    return mask

def segment_plant(image):
    mask = create_mask_for_plant(image)
    output = cv2.bitwise_and(image, image, mask = mask)
    return output

def sharpen_image(image):
    image_blurred = cv2.GaussianBlur(image, (0, 0), 3)
    image_sharp = cv2.addWeighted(image, 1.5, image_blurred, -0.5, 0)
    return image_sharp

In [ ]:
# Test image to see the changes
image = images_per_class["Small-flowered Cranesbill"][97].astype('uint8')

image_mask = create_mask_for_plant(image)
image_segmented = segment_plant(image)
image_sharpen = sharpen_image(image_segmented)

fig, axs = plt.subplots(1, 4, figsize=(20, 20))
axs[0].imshow(image.astype('uint8'))
axs[1].imshow(image_mask)
axs[2].imshow(image_segmented)
axs[3].imshow(image_sharpen)

In [ ]:
def get_sharp_image(image):
    image = image.astype('uint8')
    image_mask = create_mask_for_plant(image)
    image_segmented = segment_plant(image)
    image_sharpen = sharpen_image(image_segmented)
    return image_sharpen

In [ ]:
X_train = []
Y_train = np.zeros((4750, 12))
categories = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent', 'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']
i = 0
for class_label in tqdm(images_per_class):
    for image in images_per_class[class_label]:
        sharpened_image = get_sharp_image(image)
        X_train.append(sharpened_image)
        Y_train[i, categories.index(class_label)] = 1
        i+=1
        
X_train = np.asarray(X_train)

In [ ]:
plt.imshow(X_train[0]), categories[np.argmax(Y_train[0])]

In [ ]:
np.save('data/preprocessed/X_train.npy', X_train)
np.save('data/preprocessed/Y_train.npy', Y_train)

In [ ]:
X_train_woseg = []
Y_train_woseg = np.zeros((4750, 12))
categories = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent', 'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']
i = 0
for class_label in tqdm(images_per_class):
    for image in images_per_class[class_label]:
        X_train_woseg.append(image)
        Y_train_woseg[i, categories.index(class_label)] = 1
        i+=1
        
X_train_woseg = np.asarray(X_train_woseg)

In [ ]:
np.save('data/preprocessed/X_train_woseg.npy', X_train_woseg)
np.save('data/preprocessed/Y_train_woseg.npy', Y_train_woseg)

## load test images and save them as npy file

In [ ]:
test = pd.read_csv('sample_submission.csv')
X_test = []
for fname in tqdm(test['file']):
    img = img_to_array(load_img('test/{}'.format(fname), target_size=(224, 224, 3)))
    X_test.append(get_sharpen_image(img))
    
X_test = np.asarray(X_test)
np.save('data/preprocessed/X_test.npy', X_test)

In [ ]:
plt.imshow(X_test[12])

In [ ]:
test = pd.read_csv('sample_submission.csv')
X_test_woseg = []
for fname in tqdm(test['file']):
    img = img_to_array(load_img('test/{}'.format(fname), target_size=(224, 224, 3)))
    X_test_woseg.append(img)
    
X_test_woseg = np.asarray(X_test_woseg)
np.save('data/preprocessed/X_test_woseg.npy', X_test_woseg)